# Migrating TensorFlow EfficientNet to Habana Gaudi<sup>TM</sup>

In this Jupyter notebook, we will learn how to migrate EfficientNet in public TensorFlow [model garden](https://github.com/tensorflow/models/tree/master/official/vision/image_classification) to Habana Gaudi<sup>TM</sup> device with very limited code changes. We will first modify an existing YAML configuration file for GPU and enable the model training on CPU. Then we will add code to the training script to load Habana software modules and enable it on HPU.

First of all, check the current directory to prepare for cloning TensorFlow model's repository.

In [ ]:
%pwd

Then, we will clone TensorFlow [models](https://github.com/tensorflow/models.git)  repository to the current directory.

In [ ]:
!git clone https://github.com/tensorflow/models.git

We need to download Habana software packages as the dependency to enable EfficientNet on HPU. So we will clone Habana [Model-References](https://github.com/HabanaAI/Model-References.git) repository branch 0.15.4 to the current directory. If you have already cloned this repository to your local directory, skip the following cell.

In [ ]:
!git clone -b 0.15.4 https://github.com/HabanaAI/Model-References.git

Verify if both repositories were cloned successfully in the current location.

In [ ]:
%ls

Set Python3.7 as executable Python for EfficientNet training since in this DLAMI, the Deep Learning packages were installed under Python3.7 location.

In [ ]:
!which python3.7

In [ ]:
%set_env PYTHON=/usr/bin/python3.7

Check if Python3.7 was set to executable Python successfully.

In [ ]:
!$PYTHON --version

Check if the current PYTHONPATH contains TensorFlow `models` location and Habana `Model-References` location.

In [ ]:
%env PYTHONPATH

If PYTHONPATH doesn't include TensorFlow `models` repository and Habana `Model-References` repository locations, then add them. The following command assumes the repositories were cloned to `/home/ubuntu/` directory. Modify it accordingly if they are located in a difference folder.

In [ ]:
%set_env PYTHONPATH=/home/ubuntu/Model-References:/home/ubuntu/models

Verify if the repository locations were added to the PYTHONPATH with the command above.

In [ ]:
%env PYTHONPATH

We will be using Keras EfficientNet at https://github.com/tensorflow/models/tree/master/official/vision/image_classification as the example to show how to enable a public model on Habana Gaudi device. 

EfficientNet is a convolutional neural network architecture and scaling method that uniformly scales all dimensions of depth/width/resolution using a compound coefficient. The model was first introduced by Tan et al. in [EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks](https://arxiv.org/abs/1905.11946).  In this session, we are going to use EfficientNet baseline model EfficientNet-B0 as the training example.

First of all, let's enable the training with synthetic data on CPU and check its performance.

In [ ]:
%cd models/official/vision/image_classification

In [ ]:
%ls

In TensorFlow `model` repository, there are only EfficientNet configuration files for GPU and TPU under `configs` directory. We need to modify an existing configuration file for GPU to enable it on CPU.

Click the following file link and modify the contents with the suggestions as below:

[configs/examples/efficientnet/imagenet/efficientnet-b0-gpu.yaml](../edit/models/official/vision/image_classification/configs/examples/efficientnet/imagenet/efficientnet-b0-gpu.yaml)
   * Line 6:  change distribution_strategy to `off`
   * Line 7:  change num_gpus to `0`
   * Line 11: change builder to `synthetic`
   * Line 23: change builder to `synthetic`
   * Line 50: insert `steps: 1000`
   * Line 51: change epochs to `1`
   * Line 53: insert `skip_eval: True`
   
Save the file.

The modified configuration file looks as below:

<img src="enet_config.png" alt="efficientnet_config" align="left" width="700"/>


After we modify the EfficientNet configuration file above, we can run the following command to launch the training on CPU for 1000 iterations. We will skip evaluations in order to focus on training. Check the throughput for performance in the output log.

In [ ]:
!$PYTHON classifier_trainer.py --mode=train_and_eval --model_type=efficientnet --dataset=imagenet --model_dir=$HOME/log_cpu --data_dir=$HOME --config_file=configs/examples/efficientnet/imagenet/efficientnet-b0-gpu.yaml

From the output log above, we can see that the throughput for EfficientNet-B0 training on CPU with synthetic data is around `42 examples/sec`.

Now, let's modify the traning script and enable the model on Habana Gaudi device.

Click [classifier_trainer.py](../edit/models/official/vision/image_classification/classifier_trainer.py) link and edit it according to the instructions as below.

   * Line 443:  Insert the following 3 lines of code:
   ```
   from TensorFlow.common.library_loader import load_habana_module
log_info_devices = load_habana_module()
logging.info('Devices:\n%s', log_info_devices)
   ```
   
Save the file.


The modified training script looks as below:

<img src="enet_script.png" alt="efficientnet_script" align="left" width="600"/>


The 3 lines code above will load Habana software modules so that Habana Gaudi device will be aquired in the beginning of workload running, and EfficientNet training can be deployed and performed on Habana Gaudi device. This is all you need to do to enable EfficientNet on HPU.

Now, run the following command to launch the training on HPU and check the performance in the output.

In [ ]:
!$PYTHON classifier_trainer.py --mode=train_and_eval --model_type=efficientnet --dataset=imagenet --model_dir=$HOME/log_hpu --data_dir=$HOME --config_file=configs/examples/efficientnet/imagenet/efficientnet-b0-gpu.yaml

From the output log above, we can see that the throughput for EfficientNet-B0 training on Habana Gaudi with synthetic data is around `360 examples/sec`.